In [1]:
import random
import os
import os.path as osp
import re
import pickle
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import find_peaks
import seaborn as sns
import pickle
import torch
import glob
from scipy.stats import entropy
from collections import defaultdict, Counter

pd.set_option('display.expand_frame_repr', False)  # DataFrameを改行せずに表示
pd.set_option('display.max_columns', None)  # すべての列を表示

In [2]:
def extract_finalQP(filename):
    match = re.search(r'2ndQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    match = re.search(r'1stQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    return None


def extract_1stQP(filename):
    match = re.search(r'1stQP(\d+)', filename)
    if match:
        return int(match.group(1))
    
    return None


def ratio_double_compressed(mean_difference, final_QP):
    # mean_difference = mean_difference[0]
    # final_QP = final_QP[0]
    clamped_mean_difference = np.maximum(mean_difference, -0.01)
    
    #全体のエネルギーを計算
    energy = np.sum(np.square(clamped_mean_difference))
    # energy = np.sum(np.square(mean_difference))
    
    #QP2より右側のエネルギーを計算
    right_energy = np.sum(np.square(clamped_mean_difference[final_QP+1:52]))

        
    # エネルギー比を計算して閾値と比較
    if energy > 0:
        return right_energy / energy
    
    else:
        return 0

    
def is_double_compressed(mean_difference, final_QP, threshold):
    mean_difference = mean_difference[0]
    final_QP = final_QP[0]
    clamped_mean_difference = np.maximum(mean_difference, -0.01)
    
    #全体のエネルギーを計算
    energy = np.sum(np.square(clamped_mean_difference))
    # energy = np.sum(np.square(mean_difference))
    
    #QP2より右側のエネルギーを計算
    right_energy = np.sum(np.square(clamped_mean_difference[final_QP+1:52]))
    
    # エネルギー比を計算して閾値と比較
    if energy <= 0:
        return -1
    
    elif (right_energy / energy) != 0 and (right_energy / energy) > threshold:
        return True
    
    elif (right_energy / energy) != 0 and (right_energy / energy) <= threshold:
        return False
    
    else:
        return -1

def calculate_mae(file_path):
    try:
        with open(file_path, 'rb') as file:
            loaded_data, loaded_data_shifted = pickle.load(file)
    except Exception as e:
        print(f"Error occurred while loading {file_path}: {e}")
        return None

    # タプル内のリストを抽出
    original_mae = np.array(loaded_data)
    shifted_mae = np.array(loaded_data_shifted)

    # Coding ghostを計算してリストに格納する
    mae_difference = shifted_mae - original_mae
    
    # mae_differenceの各要素においてマイナスの値を0に変換
    # mae_difference_positive = np.maximum(mae_difference, 0)
    
    return mae_difference

In [3]:
rootpath_csv = "/Prove/Yoshihisa/HEIF_ghost/HEIF_IMAGES_CSV/"
rootpath_pkl = "/Prove/Yoshihisa/HEIF_ghost/PKL/"

train_list1 = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30"]
train_list2 = ["31", "32", "33", "34", "35", "36", "37", "38", "39", "40", "41", "42", "43", "44", "45", "46", "47", "48", "49", "50", "51", "52", "53", "54", "55", "56", "57", "58", "59", "60"]
train_list3 = ["61", "62", "63", "64", "65", "66", "67", "68", "69", "70", "71", "72", "73", "74", "75", "76", "77", "78", "79", "80", "81", "82", "83", "84", "85", "86", "87", "88", "89", "90"]
train_list4 = ["91", "92", "93", "94", "95", "96", "97", "98", "99", "100", "101", "102", "103", "104", "105", "106", "107", "108", "109", "110", "111", "112", "113", "114", "115", "116", "117", "118", "119", "120"]
train_list5 = ["121", "122", "123", "124", "125", "126", "127", "128", "129", "130", "131", "132", "133", "134", "135", "136", "137", "138", "139", "140", "141", "142", "143", "144", "145", "146", "147", "148", "149", "150"]
train_list6 = ["151", "152", "153", "154", "155", "156", "157", "158", "159", "160", "161", "162", "163", "164", "165", "166", "167", "168", "169", "170", "171", "172", "173", "174", "175", "176", "177", "178", "179", "180"]
train_list7 = ["181", "182", "183", "184", "185", "186", "187", "188", "189", "190", "191", "192", "193", "194", "195", "196", "197", "198", "199", "200", "201", "202", "203", "204", "205", "206", "207", "208", "209", "210"]
train_list8 = ["211", "212", "213", "214", "215", "216", "217", "218", "219", "220", "221", "222", "223", "224", "225", "226", "227", "228", "229", "230", "231", "232", "233", "234", "235", "236", "237", "238", "239", "240"]
train_list9 = ["241", "242", "243", "244", "245", "246", "247", "248", "249", "250", "251", "252", "253", "254", "255", "256", "257", "258", "259", "260", "261", "262", "263", "264", "265", "266", "267", "268", "269", "270"]
train_list10 = ["271", "272", "273", "274", "275", "276", "277", "278", "279", "280", "281", "282", "283", "284", "285", "286", "287", "288", "289", "290", "291", "292", "293", "294", "295", "296", "297", "298", "299", "300"]

all_train_lists = [train_list1, train_list2, train_list3, train_list4, train_list5,
                   train_list6, train_list7, train_list8, train_list9, train_list10]

# すべてのリストを1つのリストに結合する
combined_train_list = sum(all_train_lists, [])

# リストの順序をランダムにシャッフルする
random.shuffle(combined_train_list)

# シャッフルされたリストを10個のグループに分割する
train_lists = [combined_train_list[i:i+30] for i in range(0, len(combined_train_list), 30)]
print(train_lists)



# CSV関連のリストを生成
csv_single_listsA = [[] for _ in range(10)]
csv_single_recompress_listsA = [[] for _ in range(10)]
csv_second_largeQP1_listsA = [[] for _ in range(10)]
csv_second_recompress_largeQP1_listsA = [[] for _ in range(10)]
csv_second_sameQP_listsA = [[] for _ in range(10)]
csv_second_recompress_sameQP_listsA = [[] for _ in range(10)]
csv_second_largeQP2_listsA = [[] for _ in range(10)]
csv_second_recompress_largeQP2_listsA = [[] for _ in range(10)]

def process_csv_lists(rootpath, train_list, single_list, single_recompress_list, 
                      second_largeQP1_list, second_recompress_largeQP1_list, 
                      second_sameQP_list, second_recompress_sameQP_list,
                      second_largeQP2_list, second_recompress_largeQP2_list):
    
    for image in train_list:
        single_path = osp.join(rootpath, f'HEIF_images_single_csv/{image}_*')
        single_recompress_path = osp.join(rootpath, f'HEIF_images_second_sameQP_csv/{image}_*')
        
        second_largeQP1_path = osp.join(rootpath, f'HEIF_images_second_csv/{image}_*')
        second_recompress_largeQP1_path = osp.join(rootpath, f'HEIF_images_triple_csv/{image}_*')
        
        second_sameQP_path = osp.join(rootpath, f'HEIF_images_second_sameQP_csv/{image}_*')
        second_recompress_sameQP_path = osp.join(rootpath, f'HEIF_images_triple_sameQP_csv/{image}_*')
        
        second_largeQP2_path = osp.join(rootpath, f'HEIF_images_second_largeQP_csv/{image}_*')
        second_recompress_largeQP2_path = osp.join(rootpath, f'HEIF_images_triple_largeQP_csv/{image}_*')
        
        for path in sorted(glob.glob(single_path)):
            single_list.append(path)
        for path in sorted(glob.glob(single_recompress_path)):
            single_recompress_list.append(path)
        for path in sorted(glob.glob(second_largeQP1_path)):
            second_largeQP1_list.append(path)
        for path in sorted(glob.glob(second_recompress_largeQP1_path)):
            second_recompress_largeQP1_list.append(path)
        for path in sorted(glob.glob(second_sameQP_path)):
            second_sameQP_list.append(path)
        for path in sorted(glob.glob(second_recompress_sameQP_path)):
            second_recompress_sameQP_list.append(path)
        for path in sorted(glob.glob(second_largeQP2_path)):
            second_largeQP2_list.append(path)
        for path in sorted(glob.glob(second_recompress_largeQP2_path)):
            second_recompress_largeQP2_list.append(path)

# 各カテゴリのCSVリストを生成
for train_list, single_list, single_recompress_list in zip(train_lists, 
                                                           csv_single_listsA,
                                                           csv_single_recompress_listsA):
    process_csv_lists(rootpath_csv, train_list, single_list, single_recompress_list, 
                      [], [], [], [], [], [])


for train_list, second_largeQP1_list, second_recompress_largeQP1_list, second_sameQP_list, second_recompress_sameQP_list, second_largeQP2_list, second_recompress_largeQP2_list in zip(train_lists, 
                                                                                                                                                                                                                   csv_second_largeQP1_listsA,
                                                                                                                                                                                                                   csv_second_recompress_largeQP1_listsA,
                                                                                                                                                                                                                   csv_second_sameQP_listsA,
                                                                                                                                                                                                                   csv_second_recompress_sameQP_listsA,
                                                                                                                                                                                                                   csv_second_largeQP2_listsA,
                                                                                                                                                                                                                   csv_second_recompress_largeQP2_listsA):
    process_csv_lists(rootpath_csv, train_list, [], [], 
                      second_largeQP1_list, second_recompress_largeQP1_list, 
                      second_sameQP_list, second_recompress_sameQP_list,
                      second_largeQP2_list, second_recompress_largeQP2_list)

    

    
# 出力リストを初期化
pkl_single_listsA = [[] for _ in range(10)]
pkl_single_recompress_listsA = [[] for _ in range(10)]
pkl_second_largeQP1_listsA = [[] for _ in range(10)]
pkl_second_recompress_largeQP1_listsA = [[] for _ in range(10)]
pkl_second_sameQP_listsA = [[] for _ in range(10)]
pkl_second_recompress_sameQP_listsA = [[] for _ in range(10)]
pkl_second_largeQP2_listsA = [[] for _ in range(10)]
pkl_second_recompress_largeQP2_listsA = [[] for _ in range(10)]    

def process_train_lists_pkl(rootpath, train_list, single_list, single_recompress_list, 
                            second_largeQP1_list, second_recompress_largeQP1_list, 
                            second_sameQP_list, second_recompress_sameQP_list,
                            second_largeQP2_list, second_recompress_largeQP2_list):
    
    for image in train_list:
        single_path = osp.join(rootpath, f'pkl_single/{image}_*')
        single_recompress_path = osp.join(rootpath, f'pkl_second_sameQP/{image}_*')
        
        second_largeQP1_path = osp.join(rootpath, f'pkl_second/{image}_*')
        second_recompress_largeQP1_path = osp.join(rootpath, f'pkl_triple/{image}_*')
        
        second_sameQP_path = osp.join(rootpath, f'pkl_second_sameQP/{image}_*')
        second_recompress_sameQP_path = osp.join(rootpath, f'pkl_triple_sameQP/{image}_*')
        
        second_largeQP2_path = osp.join(rootpath, f'pkl_second_largeQP/{image}_*')
        second_recompress_largeQP2_path = osp.join(rootpath, f'pkl_triple_largeQP/{image}_*')
        

        for path in sorted(glob.glob(single_path)):
            single_list.append(path)
        for path in sorted(glob.glob(single_recompress_path)):
            single_recompress_list.append(path)
            
        for path in sorted(glob.glob(second_largeQP1_path)):
            second_largeQP1_list.append(path)
        for path in sorted(glob.glob(second_recompress_largeQP1_path)):
            second_recompress_largeQP1_list.append(path)
                
        for path in sorted(glob.glob(second_sameQP_path)):
            second_sameQP_list.append(path)
        for path in sorted(glob.glob(second_recompress_sameQP_path)):
            second_recompress_sameQP_list.append(path)
            
        for path in sorted(glob.glob(second_largeQP2_path)):
            second_largeQP2_list.append(path)
        for path in sorted(glob.glob(second_recompress_largeQP2_path)):
            second_recompress_largeQP2_list.append(path)

# 各カテゴリのリストを生成
for train_list, single_list, single_recompress_list in zip(train_lists, 
                                                           pkl_single_listsA,
                                                           pkl_single_recompress_listsA):
    process_train_lists_pkl(rootpath_pkl, train_list, single_list, single_recompress_list, 
                            [], [], [], [], [], [])


for train_list, second_largeQP1_list, second_recompress_largeQP1_list, second_sameQP_list, second_recompress_sameQP_list, second_largeQP2_list, second_recompress_largeQP2_list in zip(train_lists, 
                                                                                                                                                                                                                   pkl_second_largeQP1_listsA,
                                                                                                                                                                                                                   pkl_second_recompress_largeQP1_listsA,
                                                                                                                                                                                                                   pkl_second_sameQP_listsA,
                                                                                                                                                                                                                   pkl_second_recompress_sameQP_listsA,
                                                                                                                                                                                                                   pkl_second_largeQP2_listsA,
                                                                                                                                                                                                                   pkl_second_recompress_largeQP2_listsA):
    process_train_lists_pkl(rootpath_pkl, train_list, [], [], 
                            second_largeQP1_list, second_recompress_largeQP1_list, 
                            second_sameQP_list, second_recompress_sameQP_list,
                            second_largeQP2_list, second_recompress_largeQP2_list)


print("\nCSV Single ListsA:")
for i, lst in enumerate(csv_single_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

print("\nCSV Single Recompress ListsA:")
for i, lst in enumerate(csv_single_recompress_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

print("\nCSV Second Large QP1 ListsA:")
for i, lst in enumerate(csv_second_largeQP1_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

print("\nCSV Second Recompress Large QP1 ListsA:")
for i, lst in enumerate(csv_second_recompress_largeQP1_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

print("\nCSV Second Same QP ListsA:")
for i, lst in enumerate(csv_second_sameQP_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

print("\nCSV Second Recompress Same QP ListsA:")
for i, lst in enumerate(csv_second_recompress_sameQP_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

print("\nCSV Second Large QP2 ListsA:")
for i, lst in enumerate(csv_second_largeQP2_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

print("\nCSV Second Recompress Large QP2 ListsA:")
for i, lst in enumerate(csv_second_recompress_largeQP2_listsA, 1):
    print(f"CSV List {i}A: {len(lst)}")

# 出力リストを表示
print("\nPKL Single ListsA:")
for i, lst in enumerate(pkl_single_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

print("\nPKL Single Recompress ListsA:")
for i, lst in enumerate(pkl_single_recompress_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

print("\nPKL Second Large QP1 ListsA:")
for i, lst in enumerate(pkl_second_largeQP1_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

print("\nPKL Second Recompress Large QP1 ListsA:")
for i, lst in enumerate(pkl_second_recompress_largeQP1_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

print("\nPKL Second Same QP ListsA:")
for i, lst in enumerate(pkl_second_sameQP_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

print("\nPKL Second Recompress Same QP ListsA:")
for i, lst in enumerate(pkl_second_recompress_sameQP_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

print("\nPKL Second Large QP2 ListsA:")
for i, lst in enumerate(pkl_second_largeQP2_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

print("\nPKL Second Recompress Large QP2 ListsA:")
for i, lst in enumerate(pkl_second_recompress_largeQP2_listsA, 1):
    print(f"PKL List {i}A: {len(lst)}")

[['48', '2', '17', '44', '62', '176', '37', '67', '242', '119', '184', '163', '253', '99', '115', '84', '97', '209', '125', '252', '149', '198', '225', '109', '222', '86', '288', '208', '274', '275'], ['104', '271', '132', '53', '88', '200', '152', '51', '27', '160', '250', '43', '92', '121', '29', '279', '265', '205', '193', '105', '202', '155', '295', '248', '283', '98', '4', '243', '117', '185'], ['5', '167', '221', '277', '3', '72', '124', '11', '113', '229', '299', '12', '291', '141', '296', '254', '240', '159', '256', '6', '236', '13', '91', '210', '26', '173', '64', '192', '68', '95'], ['169', '57', '273', '14', '172', '102', '170', '298', '280', '258', '197', '38', '73', '227', '110', '164', '41', '145', '50', '129', '65', '257', '219', '223', '28', '83', '146', '156', '187', '137'], ['218', '108', '23', '126', '8', '42', '63', '259', '31', '18', '284', '103', '270', '266', '45', '74', '111', '180', '19', '133', '89', '122', '93', '61', '138', '175', '290', '244', '234', '75'],

In [4]:
# sameQP = 1
QPD0 = ["_1stQP5_2ndQP5", "_1stQP10_2ndQP10", "_1stQP16_2ndQP16", "_1stQP20_2ndQP20", "_1stQP24_2ndQP24", 
        "_1stQP27_2ndQP27", "_1stQP32_2ndQP32", "_1stQP39_2ndQP39", "_1stQP42_2ndQP42", "_1stQP45_2ndQP45"]

# largeQP1 = 30
QPD1 = ["_1stQP25_2ndQP24_", "_1stQP40_2ndQP39_"]
QPD3 = ["_1stQP30_2ndQP27_", "_1stQP35_2ndQP32_", "_1stQP45_2ndQP42_"]
QPD4 = ["_1stQP20_2ndQP16_"]
QPD5 = ["_1stQP10_2ndQP5_", "_1stQP15_2ndQP10_", "_1stQP25_2ndQP20_", "_1stQP32_2ndQP27_", "_1stQP50_2ndQP45_"]
QPD6 = ["_1stQP30_2ndQP24_", "_1stQP45_2ndQP39_"]
QPD8 = ["_1stQP32_2ndQP24_", "_1stQP35_2ndQP27_", "_1stQP40_2ndQP32_", "_1stQP50_2ndQP42_"]
QPD9 = ["_1stQP25_2ndQP16_"]
QPD10 = ["_1stQP15_2ndQP5_", "_1stQP20_2ndQP10_", "_1stQP30_2ndQP20_"]
QPD11 = ["_1stQP35_2ndQP24_", "_1stQP50_2ndQP39_"]
QPD12 = ["_1stQP32_2ndQP20_"]
QPD13 = ["_1stQP40_2ndQP27_", "_1stQP45_2ndQP32_"]
QPD14 = ["_1stQP30_2ndQP16_"]
QPD15 = ["_1stQP20_2ndQP5_", "_1stQP25_2ndQP10_", "_1stQP35_2ndQP20_"]
QPD16 = ["_1stQP32_2ndQP16_", "_1stQP40_2ndQP24_"]
QPD18 = ["_1stQP45_2ndQP27_", "_1stQP50_2ndQP32_"]
QPD19 = ["_1stQP35_2ndQP16_"]
QPD20 = ["_1stQP25_2ndQP5_", "_1stQP30_2ndQP10_", "_1stQP40_2ndQP20_"]
QPD21 = ["_1stQP45_2ndQP24_"]
QPD22 = ["_1stQP32_2ndQP10_"]
QPD23 = ["_1stQP50_2ndQP27_"]
QPD24 = ["_1stQP40_2ndQP16_"]
QPD25 = ["_1stQP30_2ndQP5_", "_1stQP35_2ndQP10_", "_1stQP45_2ndQP20_"]
QPD26 = ["_1stQP50_2ndQP24_"]
QPD27 = ["_1stQP32_2ndQP5_"]
QPD29 = ["_1stQP45_2ndQP16_"]
QPD30 = ["_1stQP35_2ndQP5_", "_1stQP40_2ndQP10_", "_1stQP50_2ndQP20_"]
QPD34 = ["_1stQP50_2ndQP16_"]
QPD35 = ["_1stQP40_2ndQP5_", "_1stQP45_2ndQP10_"]
QPD40 = ["_1stQP45_2ndQP5_", "_1stQP50_2ndQP10_"]
QPD45 = ["_1stQP50_2ndQP5_"]

# largeQP2 = 23
QPD1M = ["_1stQP15_2ndQP16"]
QPD2M = ["_1stQP25_2ndQP27", "_1stQP30_2ndQP32", "_1stQP40_2ndQP42"]
QPD4M = ["_1stQP20_2ndQP24", "_1stQP35_2ndQP39"]
QPD5M = ["_1stQP15_2ndQP20", "_1stQP40_2ndQP45"]
QPD6M = ["_1stQP10_2ndQP16"]
QPD7M = ["_1stQP20_2ndQP27", "_1stQP25_2ndQP32", "_1stQP32_2ndQP39", "_1stQP35_2ndQP42"]
QPD9M = ["_1stQP15_2ndQP24", "_1stQP30_2ndQP39"]
QPD10M = ["_1stQP10_2ndQP20", "_1stQP32_2ndQP42", "_1stQP35_2ndQP45"]
QPD12M = ["_1stQP15_2ndQP27", "_1stQP20_2ndQP32", "_1stQP30_2ndQP42"]
QPD13M = ["_1stQP32_2ndQP45"]
QPD14M = ["_1stQP10_2ndQP24", "_1stQP25_2ndQP39"]
QPD15M = ["_1stQP30_2ndQP45"]
QPD17M = ["_1stQP10_2ndQP27", "_1stQP15_2ndQP32", "_1stQP25_2ndQP42"]
QPD19M = ["_1stQP20_2ndQP39"]
QPD20M = ["_1stQP25_2ndQP45"]
QPD22M = ["_1stQP10_2ndQP32", "_1stQP20_2ndQP42"]
QPD24M = ["_1stQP15_2ndQP39"]
QPD25M = ["_1stQP20_2ndQP45"]
QPD27M = ["_1stQP15_2ndQP42"]
QPD29M = ["_1stQP10_2ndQP39"]
QPD30M = ["_1stQP15_2ndQP45"]
QPD32M = ["_1stQP10_2ndQP42"]
QPD35M = ["_1stQP10_2ndQP45"]

In [5]:
# single_listsおよびsingle_recompress_listsは初期化されている前提
single_csv1 = list(zip(csv_single_listsA[0], pkl_single_listsA[0], csv_single_recompress_listsA[0], pkl_single_recompress_listsA[0]))
single_csv2 = list(zip(csv_single_listsA[1], pkl_single_listsA[1], csv_single_recompress_listsA[1], pkl_single_recompress_listsA[1]))
single_csv3 = list(zip(csv_single_listsA[2], pkl_single_listsA[2], csv_single_recompress_listsA[2], pkl_single_recompress_listsA[2]))
single_csv4 = list(zip(csv_single_listsA[3], pkl_single_listsA[3], csv_single_recompress_listsA[3], pkl_single_recompress_listsA[3]))
single_csv5 = list(zip(csv_single_listsA[4], pkl_single_listsA[4], csv_single_recompress_listsA[4], pkl_single_recompress_listsA[4]))
single_csv6 = list(zip(csv_single_listsA[5], pkl_single_listsA[5], csv_single_recompress_listsA[5], pkl_single_recompress_listsA[5]))
single_csv7 = list(zip(csv_single_listsA[6], pkl_single_listsA[6], csv_single_recompress_listsA[6], pkl_single_recompress_listsA[6]))
single_csv8 = list(zip(csv_single_listsA[7], pkl_single_listsA[7], csv_single_recompress_listsA[7], pkl_single_recompress_listsA[7]))
single_csv9 = list(zip(csv_single_listsA[8], pkl_single_listsA[8], csv_single_recompress_listsA[8], pkl_single_recompress_listsA[8]))
single_csv10 = list(zip(csv_single_listsA[9], pkl_single_listsA[9], csv_single_recompress_listsA[9], pkl_single_recompress_listsA[9]))
print('single images: ', len(single_csv9))

single images:  300


In [6]:
# Large_QP1
second_largeQP1_csv1 = list(zip(csv_second_largeQP1_listsA[0], pkl_second_largeQP1_listsA[0], csv_second_recompress_largeQP1_listsA[0], pkl_second_recompress_largeQP1_listsA[0]))
second_largeQP1_csv2 = list(zip(csv_second_largeQP1_listsA[1], pkl_second_largeQP1_listsA[1], csv_second_recompress_largeQP1_listsA[1], pkl_second_recompress_largeQP1_listsA[1]))
second_largeQP1_csv3 = list(zip(csv_second_largeQP1_listsA[2], pkl_second_largeQP1_listsA[2], csv_second_recompress_largeQP1_listsA[2], pkl_second_recompress_largeQP1_listsA[2]))
second_largeQP1_csv4 = list(zip(csv_second_largeQP1_listsA[3], pkl_second_largeQP1_listsA[3], csv_second_recompress_largeQP1_listsA[3], pkl_second_recompress_largeQP1_listsA[3]))
second_largeQP1_csv5 = list(zip(csv_second_largeQP1_listsA[4], pkl_second_largeQP1_listsA[4], csv_second_recompress_largeQP1_listsA[4], pkl_second_recompress_largeQP1_listsA[4]))
second_largeQP1_csv6 = list(zip(csv_second_largeQP1_listsA[5], pkl_second_largeQP1_listsA[5], csv_second_recompress_largeQP1_listsA[5], pkl_second_recompress_largeQP1_listsA[5]))
second_largeQP1_csv7 = list(zip(csv_second_largeQP1_listsA[6], pkl_second_largeQP1_listsA[6], csv_second_recompress_largeQP1_listsA[6], pkl_second_recompress_largeQP1_listsA[6]))
second_largeQP1_csv8 = list(zip(csv_second_largeQP1_listsA[7], pkl_second_largeQP1_listsA[7], csv_second_recompress_largeQP1_listsA[7], pkl_second_recompress_largeQP1_listsA[7]))
second_largeQP1_csv9 = list(zip(csv_second_largeQP1_listsA[8], pkl_second_largeQP1_listsA[8], csv_second_recompress_largeQP1_listsA[8], pkl_second_recompress_largeQP1_listsA[8]))
second_largeQP1_csv10 = list(zip(csv_second_largeQP1_listsA[9], pkl_second_largeQP1_listsA[9], csv_second_recompress_largeQP1_listsA[9], pkl_second_recompress_largeQP1_listsA[9]))
# print((second_largeQP1_csv1))


# Define the datasets
datasets = [
    second_largeQP1_csv1, second_largeQP1_csv2, second_largeQP1_csv3,
    second_largeQP1_csv4, second_largeQP1_csv5, second_largeQP1_csv6,
    second_largeQP1_csv7, second_largeQP1_csv8, second_largeQP1_csv9,
    second_largeQP1_csv10
]

# Define QPD lists
QPD = {
    "QPD1": ["_1stQP25_2ndQP24_", "_1stQP40_2ndQP39_"],
    "QPD3": ["_1stQP30_2ndQP27_", "_1stQP35_2ndQP32_", "_1stQP45_2ndQP42_"],
    "QPD4": ["_1stQP20_2ndQP16_"],
    "QPD5": ["_1stQP10_2ndQP5_", "_1stQP15_2ndQP10_", "_1stQP25_2ndQP20_", "_1stQP32_2ndQP27_", "_1stQP50_2ndQP45_"],
    "QPD6": ["_1stQP30_2ndQP24_", "_1stQP45_2ndQP39_"],
    "QPD8": ["_1stQP32_2ndQP24_", "_1stQP35_2ndQP27_", "_1stQP40_2ndQP32_", "_1stQP50_2ndQP42_"],
    "QPD9": ["_1stQP25_2ndQP16_"],
    "QPD10": ["_1stQP15_2ndQP5_", "_1stQP20_2ndQP10_", "_1stQP30_2ndQP20_"],
    "QPD11": ["_1stQP35_2ndQP24_", "_1stQP50_2ndQP39_"],
    "QPD12": ["_1stQP32_2ndQP20_"],
    "QPD13": ["_1stQP40_2ndQP27_", "_1stQP45_2ndQP32_"],
    "QPD14": ["_1stQP30_2ndQP16_"],
    "QPD15": ["_1stQP20_2ndQP5_", "_1stQP25_2ndQP10_", "_1stQP35_2ndQP20_"],
    "QPD16": ["_1stQP32_2ndQP16_", "_1stQP40_2ndQP24_"],
    "QPD18": ["_1stQP45_2ndQP27_", "_1stQP50_2ndQP32_"],
    "QPD19": ["_1stQP35_2ndQP16_"],
    "QPD20": ["_1stQP25_2ndQP5_", "_1stQP30_2ndQP10_", "_1stQP40_2ndQP20_"],
    "QPD21": ["_1stQP45_2ndQP24_"],
    "QPD22": ["_1stQP32_2ndQP10_"],
    "QPD23": ["_1stQP50_2ndQP27_"],
    "QPD24": ["_1stQP40_2ndQP16_"],
    "QPD25": ["_1stQP30_2ndQP5_", "_1stQP35_2ndQP10_", "_1stQP45_2ndQP20_"],
    "QPD26": ["_1stQP50_2ndQP24_"],
    "QPD27": ["_1stQP32_2ndQP5_"],
    "QPD29": ["_1stQP45_2ndQP16_"],
    "QPD30": ["_1stQP35_2ndQP5_", "_1stQP40_2ndQP10_", "_1stQP50_2ndQP20_"],
    "QPD34": ["_1stQP50_2ndQP16_"],
    "QPD35": ["_1stQP40_2ndQP5_", "_1stQP45_2ndQP10_"],
    "QPD40": ["_1stQP45_2ndQP5_", "_1stQP50_2ndQP10_"],
    "QPD45": ["_1stQP50_2ndQP5_"]
}

# Priority QPD lists
priority_qpd = {"QPD1", "QPD3", "QPD4", "QPD5", "QPD6", "QPD8", "QPD9", "QPD10"}

# Create a function to check which QPD lists a string belongs to
def check_qpd_lists(s):
    qpd_lists = []
    for k, v in QPD.items():
        if any(qp in s for qp in v):
            qpd_lists.append(k)
    return qpd_lists

# Initialize the selected data list
selected_data = [[] for _ in range(len(datasets))]

# Step 1: Select 6 items from each priority QPD from each dataset
for dataset_idx, dataset in enumerate(datasets):
    random.shuffle(dataset)
    selected_counts = defaultdict(int)
    
    for item in dataset:
        item_str = item[0]  # Assume the first element in the tuple is the string to check
        qpd_lists = check_qpd_lists(item_str)
        priority_check = any(qpd in priority_qpd for qpd in qpd_lists)
        
        if priority_check:
            if any(selected_counts[qpd] < 6 for qpd in qpd_lists if qpd in priority_qpd):
                selected_data[dataset_idx].append(item)
                for qpd in qpd_lists:
                    if qpd in priority_qpd:
                        selected_counts[qpd] += 1
                        
        if all(selected_counts[qpd] >= 6 for qpd in priority_qpd):
            break

# Step 2: Select exactly 2 items from each non-priority QPD
non_priority_qpd = set(QPD.keys()) - priority_qpd

for dataset_idx, dataset in enumerate(datasets):
    random.shuffle(dataset)
    current_selected = selected_data[dataset_idx]
    non_priority_counts = defaultdict(int)
    remaining_qpd_items = {qpd: [] for qpd in non_priority_qpd}
    
    for item in dataset:
        item_str = item[0]  # Assume the first element in the tuple is the string to check
        qpd_lists = check_qpd_lists(item_str)
        non_priority_check = any(qpd in non_priority_qpd for qpd in qpd_lists)
        
        if non_priority_check:
            for qpd in qpd_lists:
                if qpd in non_priority_qpd and non_priority_counts[qpd] < 2:
                    remaining_qpd_items[qpd].append(item)
                    non_priority_counts[qpd] += 1
    
    for qpd, items in remaining_qpd_items.items():
        current_selected.extend(items[:2])
    
    selected_data[dataset_idx] = current_selected

# Step 3: Select 8 more items from non-priority QPD to make 100 items
for dataset_idx, dataset in enumerate(datasets):
    random.shuffle(dataset)
    current_selected = selected_data[dataset_idx]
    extra_items_needed = 100 - len(current_selected)
    
    if extra_items_needed > 0:
        remaining_qpd_items = {qpd: [] for qpd in non_priority_qpd}
        non_priority_counts = defaultdict(int)
        
        for item in dataset:
            item_str = item[0]
            qpd_lists = check_qpd_lists(item_str)
            non_priority_check = any(qpd in non_priority_qpd for qpd in qpd_lists)
            
            if non_priority_check:
                for qpd in qpd_lists:
                    if qpd in non_priority_qpd and non_priority_counts[qpd] < 3:
                        remaining_qpd_items[qpd].append(item)
                        non_priority_counts[qpd] += 1
        
        extra_items_selected = 0
        while extra_items_selected < extra_items_needed:
            for qpd, items in remaining_qpd_items.items():
                if items and extra_items_selected < extra_items_needed:
                    current_selected.append(items.pop(0))
                    extra_items_selected += 1
    
    selected_data[dataset_idx] = current_selected[:100]
    
# Check the distribution of QPDs
qpd_distribution = Counter()
for data_list in selected_data:
    for item in data_list:
        item_str = item[0]
        qpd_lists = check_qpd_lists(item_str)
        for qpd in qpd_lists:
            qpd_distribution[qpd] += 1

print("\nQPD Distribution:\n", qpd_distribution)

second_largeQP1_csv1 = selected_data[0]
second_largeQP1_csv2 = selected_data[1]
second_largeQP1_csv3 = selected_data[2]
second_largeQP1_csv4 = selected_data[3]
second_largeQP1_csv5 = selected_data[4]
second_largeQP1_csv6 = selected_data[5]
second_largeQP1_csv7 = selected_data[6]
second_largeQP1_csv8 = selected_data[7]
second_largeQP1_csv9 = selected_data[8]
second_largeQP1_csv10 = selected_data[9]
print('\ndouble images largeQP1: ', len(second_largeQP1_csv1))


QPD Distribution:
 Counter({'QPD8': 60, 'QPD5': 60, 'QPD1': 60, 'QPD3': 60, 'QPD10': 60, 'QPD4': 60, 'QPD6': 60, 'QPD9': 60, 'QPD19': 30, 'QPD34': 30, 'QPD12': 30, 'QPD35': 30, 'QPD23': 30, 'QPD11': 30, 'QPD15': 30, 'QPD22': 30, 'QPD25': 20, 'QPD13': 20, 'QPD20': 20, 'QPD29': 20, 'QPD18': 20, 'QPD14': 20, 'QPD21': 20, 'QPD45': 20, 'QPD24': 20, 'QPD16': 20, 'QPD40': 20, 'QPD27': 20, 'QPD26': 20, 'QPD30': 20})

double images largeQP1:  100


In [7]:
# sameQP
second_sameQP_csv1 = list(zip(csv_second_sameQP_listsA[0], pkl_second_sameQP_listsA[0], csv_second_recompress_sameQP_listsA[0], pkl_second_recompress_sameQP_listsA[0]))
second_sameQP_csv2 = list(zip(csv_second_sameQP_listsA[1], pkl_second_sameQP_listsA[1], csv_second_recompress_sameQP_listsA[1], pkl_second_recompress_sameQP_listsA[1]))
second_sameQP_csv3 = list(zip(csv_second_sameQP_listsA[2], pkl_second_sameQP_listsA[2], csv_second_recompress_sameQP_listsA[2], pkl_second_recompress_sameQP_listsA[2]))
second_sameQP_csv4 = list(zip(csv_second_sameQP_listsA[3], pkl_second_sameQP_listsA[3], csv_second_recompress_sameQP_listsA[3], pkl_second_recompress_sameQP_listsA[3]))
second_sameQP_csv5 = list(zip(csv_second_sameQP_listsA[4], pkl_second_sameQP_listsA[4], csv_second_recompress_sameQP_listsA[4], pkl_second_recompress_sameQP_listsA[4]))
second_sameQP_csv6 = list(zip(csv_second_sameQP_listsA[5], pkl_second_sameQP_listsA[5], csv_second_recompress_sameQP_listsA[5], pkl_second_recompress_sameQP_listsA[5]))
second_sameQP_csv7 = list(zip(csv_second_sameQP_listsA[6], pkl_second_sameQP_listsA[6], csv_second_recompress_sameQP_listsA[6], pkl_second_recompress_sameQP_listsA[6]))
second_sameQP_csv8 = list(zip(csv_second_sameQP_listsA[7], pkl_second_sameQP_listsA[7], csv_second_recompress_sameQP_listsA[7], pkl_second_recompress_sameQP_listsA[7]))
second_sameQP_csv9 = list(zip(csv_second_sameQP_listsA[8], pkl_second_sameQP_listsA[8], csv_second_recompress_sameQP_listsA[8], pkl_second_recompress_sameQP_listsA[8]))
second_sameQP_csv10 = list(zip(csv_second_sameQP_listsA[9], pkl_second_sameQP_listsA[9], csv_second_recompress_sameQP_listsA[9], pkl_second_recompress_sameQP_listsA[9]))


# Define the datasets
datasets = [
    second_sameQP_csv1, second_sameQP_csv2, second_sameQP_csv3,
    second_sameQP_csv4, second_sameQP_csv5, second_sameQP_csv6,
    second_sameQP_csv7, second_sameQP_csv8, second_sameQP_csv9,
    second_sameQP_csv10
]

# Define QPD lists
QPD = {
    "QPD5S": ["_1stQP5_2ndQP5"],
    "QPD10S": ["_1stQP10_2ndQP10"],
    "QPD16S": ["_1stQP16_2ndQP16"],
    "QPD20S": ["_1stQP20_2ndQP20"],
    "QPD24S": ["_1stQP24_2ndQP24"],
    "QPD27S": ["_1stQP27_2ndQP27"],
    "QPD32S": ["_1stQP32_2ndQP32"],
    "QPD39S": ["_1stQP39_2ndQP39"],
    "QPD42S": ["_1stQP42_2ndQP42"],
    "QPD45S": ["_1stQP45_2ndQP45"],
}

# Create a function to check which QPD lists a string belongs to
def check_qpd_lists(s):
    qpd_lists = []
    for k, v in QPD.items():
        if any(qp in s for qp in v):
            qpd_lists.append(k)
    return qpd_lists

# Function to select 100 items from each dataset ensuring QPD distribution
def select_items(datasets):
    selected_data = []
    
    for dataset in datasets:
        indices = list(range(len(dataset)))
        random.shuffle(indices)
        selected_counts = defaultdict(int)
        selected_from_dataset = []
        
        while len(selected_from_dataset) < 100 and indices:
            idx = indices.pop()
            item = dataset[idx]
            item_str = item[0]  # Assume the first element in the tuple is the string to check
            qpd_lists = check_qpd_lists(item_str)
            if qpd_lists:
                if all(selected_counts[qpd] < 100 / len(QPD) for qpd in qpd_lists):
                    selected_from_dataset.append(item)
                    for qpd in qpd_lists:
                        selected_counts[qpd] += 1
        
        selected_data.append(selected_from_dataset)
    
    return selected_data

# Select 100 items from each dataset
selected_data = select_items(datasets)

# Verify the selected data length
for i, data_list in enumerate(selected_data):
    print(f"List {i+1}: {len(data_list)} items")

# Print the distribution of QPD lists in the selected data (optional)
qpd_distribution = Counter()
for data_list in selected_data:
    for item in data_list:
        item_str = item[0]
        qpd_lists = check_qpd_lists(item_str)
        for qpd in qpd_lists:
            qpd_distribution[qpd] += 1

print("\nQPD Distribution:", qpd_distribution)


second_sameQP_csv1 = selected_data[0]
second_sameQP_csv2 = selected_data[1]
second_sameQP_csv3 = selected_data[2]
second_sameQP_csv4 = selected_data[3]
second_sameQP_csv5 = selected_data[4]
second_sameQP_csv6 = selected_data[5]
second_sameQP_csv7 = selected_data[6]
second_sameQP_csv8 = selected_data[7]
second_sameQP_csv9 = selected_data[8]
second_sameQP_csv10 = selected_data[9]
print('\ndouble images sameQP: ', len(second_sameQP_csv1))



List 1: 100 items
List 2: 100 items
List 3: 100 items
List 4: 100 items
List 5: 100 items
List 6: 100 items
List 7: 100 items
List 8: 100 items
List 9: 100 items
List 10: 100 items

QPD Distribution: Counter({'QPD5S': 100, 'QPD20S': 100, 'QPD16S': 100, 'QPD39S': 100, 'QPD42S': 100, 'QPD27S': 100, 'QPD32S': 100, 'QPD10S': 100, 'QPD24S': 100, 'QPD45S': 100})

double images sameQP:  100


In [8]:
# Large_QP2
second_largeQP2_csv1 = list(zip(csv_second_largeQP2_listsA[0], pkl_second_largeQP2_listsA[0], csv_second_recompress_largeQP2_listsA[0], pkl_second_recompress_largeQP2_listsA[0]))
second_largeQP2_csv2 = list(zip(csv_second_largeQP2_listsA[1], pkl_second_largeQP2_listsA[1], csv_second_recompress_largeQP2_listsA[1], pkl_second_recompress_largeQP2_listsA[1]))
second_largeQP2_csv3 = list(zip(csv_second_largeQP2_listsA[2], pkl_second_largeQP2_listsA[2], csv_second_recompress_largeQP2_listsA[2], pkl_second_recompress_largeQP2_listsA[2]))
second_largeQP2_csv4 = list(zip(csv_second_largeQP2_listsA[3], pkl_second_largeQP2_listsA[3], csv_second_recompress_largeQP2_listsA[3], pkl_second_recompress_largeQP2_listsA[3]))
second_largeQP2_csv5 = list(zip(csv_second_largeQP2_listsA[4], pkl_second_largeQP2_listsA[4], csv_second_recompress_largeQP2_listsA[4], pkl_second_recompress_largeQP2_listsA[4]))
second_largeQP2_csv6 = list(zip(csv_second_largeQP2_listsA[5], pkl_second_largeQP2_listsA[5], csv_second_recompress_largeQP2_listsA[5], pkl_second_recompress_largeQP2_listsA[5]))
second_largeQP2_csv7 = list(zip(csv_second_largeQP2_listsA[6], pkl_second_largeQP2_listsA[6], csv_second_recompress_largeQP2_listsA[6], pkl_second_recompress_largeQP2_listsA[6]))
second_largeQP2_csv8 = list(zip(csv_second_largeQP2_listsA[7], pkl_second_largeQP2_listsA[7], csv_second_recompress_largeQP2_listsA[7], pkl_second_recompress_largeQP2_listsA[7]))
second_largeQP2_csv9 = list(zip(csv_second_largeQP2_listsA[8], pkl_second_largeQP2_listsA[8], csv_second_recompress_largeQP2_listsA[8], pkl_second_recompress_largeQP2_listsA[8]))
second_largeQP2_csv10 = list(zip(csv_second_largeQP2_listsA[9], pkl_second_largeQP2_listsA[9], csv_second_recompress_largeQP2_listsA[9], pkl_second_recompress_largeQP2_listsA[9]))

# Define the datasets
datasets = [
    second_largeQP2_csv1, second_largeQP2_csv2, second_largeQP2_csv3,
    second_largeQP2_csv4, second_largeQP2_csv5, second_largeQP2_csv6,
    second_largeQP2_csv7, second_largeQP2_csv8, second_largeQP2_csv9,
    second_largeQP2_csv10
]


# Define QPD lists
QPD = {
    "QPD1M": ["_1stQP15_2ndQP16"],
    "QPD2M": ["_1stQP25_2ndQP27", "_1stQP30_2ndQP32", "_1stQP40_2ndQP42"],
    "QPD4M": ["_1stQP20_2ndQP24", "_1stQP35_2ndQP39"],
    "QPD5M": ["_1stQP15_2ndQP20", "_1stQP40_2ndQP45"],
    "QPD6M": ["_1stQP10_2ndQP16"],
    "QPD7M": ["_1stQP20_2ndQP27", "_1stQP25_2ndQP32", "_1stQP32_2ndQP39", "_1stQP35_2ndQP42"],
    "QPD9M": ["_1stQP15_2ndQP24", "_1stQP30_2ndQP39"],
    "QPD10M": ["_1stQP10_2ndQP20", "_1stQP32_2ndQP42", "_1stQP35_2ndQP45"],
    "QPD12M": ["_1stQP15_2ndQP27", "_1stQP20_2ndQP32", "_1stQP30_2ndQP42"],
    "QPD13M": ["_1stQP32_2ndQP45"],
    "QPD14M": ["_1stQP10_2ndQP24", "_1stQP25_2ndQP39"],
    "QPD15M": ["_1stQP30_2ndQP45"],
    "QPD17M": ["_1stQP10_2ndQP27", "_1stQP15_2ndQP32", "_1stQP25_2ndQP42"],
    "QPD19M": ["_1stQP20_2ndQP39"],
    "QPD20M": ["_1stQP25_2ndQP45"],
    "QPD22M": ["_1stQP10_2ndQP32", "_1stQP20_2ndQP42"],
    "QPD24M": ["_1stQP15_2ndQP39"],
    "QPD25M": ["_1stQP20_2ndQP45"],
    "QPD27M": ["_1stQP15_2ndQP42"],
    "QPD29M": ["_1stQP10_2ndQP39"],
    "QPD30M": ["_1stQP15_2ndQP45"],
    "QPD32M": ["_1stQP10_2ndQP42"],
    "QPD35M": ["_1stQP10_2ndQP45"]
}

# Create a function to check which QPD lists a string belongs to
def check_qpd_lists(s):
    qpd_lists = []
    for k, v in QPD.items():
        if any(qp in s for qp in v):
            qpd_lists.append(k)
    return qpd_lists

# Function to select 100 items from each dataset ensuring QPD distribution
def select_items(datasets):
    selected_data = []
    
    for dataset in datasets:
        random.shuffle(dataset)
        selected_counts = defaultdict(int)
        selected_from_dataset = []
        
        # Select 4 items from each QPD
        for qpd in QPD.keys():
            count = 0
            for item in dataset:
                if count >= 4:
                    break
                item_str = item[0]
                if qpd in check_qpd_lists(item_str) and selected_counts[qpd] < 4:
                    selected_from_dataset.append(item)
                    selected_counts[qpd] += 1
                    count += 1
                    dataset.remove(item)
        
        # Select additional 8 items from QPD to make 100 items
        remaining_qpds = list(QPD.keys())
        extra_items_needed = 100 - len(selected_from_dataset)
        extra_counts = defaultdict(int)
        
        for item in dataset:
            if extra_items_needed <= 0:
                break
            item_str = item[0]
            qpd_lists = check_qpd_lists(item_str)
            for qpd in qpd_lists:
                if qpd in remaining_qpds and extra_counts[qpd] < 1:
                    selected_from_dataset.append(item)
                    extra_counts[qpd] += 1
                    extra_items_needed -= 1
                    dataset.remove(item)
                    break
        
        selected_data.append(selected_from_dataset[:100])
    
    return selected_data

# Select 100 items from each dataset
selected_data = select_items(datasets)

# Verify the selected data length and distribution
for i, data_list in enumerate(selected_data):
    print(f"List {i+1}: {len(data_list)} items")
    
qpd_distribution = Counter()
for data_list in selected_data:
    for item in data_list:
        item_str = item[0]
        qpd_lists = check_qpd_lists(item_str)
        for qpd in qpd_lists:
            qpd_distribution[qpd] += 1

print("\nQPD Distribution:\n", qpd_distribution)


second_largeQP2_csv1 = selected_data[0]
second_largeQP2_csv2 = selected_data[1]
second_largeQP2_csv3 = selected_data[2]
second_largeQP2_csv4 = selected_data[3]
second_largeQP2_csv5 = selected_data[4]
second_largeQP2_csv6 = selected_data[5]
second_largeQP2_csv7 = selected_data[6]
second_largeQP2_csv8 = selected_data[7]
second_largeQP2_csv9 = selected_data[8]
second_largeQP2_csv10 = selected_data[9]
print('\ndouble images largeQP2: ', len(second_largeQP2_csv1))


List 1: 100 items
List 2: 100 items
List 3: 100 items
List 4: 100 items
List 5: 100 items
List 6: 100 items
List 7: 100 items
List 8: 100 items
List 9: 100 items
List 10: 100 items

QPD Distribution:
 Counter({'QPD2M': 49, 'QPD7M': 49, 'QPD5M': 47, 'QPD9M': 47, 'QPD10M': 47, 'QPD12M': 47, 'QPD17M': 46, 'QPD22M': 46, 'QPD14M': 45, 'QPD4M': 44, 'QPD19M': 42, 'QPD25M': 42, 'QPD30M': 42, 'QPD35M': 42, 'QPD15M': 41, 'QPD20M': 41, 'QPD24M': 41, 'QPD27M': 41, 'QPD29M': 41, 'QPD1M': 40, 'QPD6M': 40, 'QPD13M': 40, 'QPD32M': 40})

double images largeQP2:  100


In [9]:
# Training_data
train_csv_list1 = single_csv1 + second_largeQP1_csv1 + second_sameQP_csv1 + second_largeQP2_csv1
train_csv_list2 = single_csv2 + second_largeQP1_csv2 + second_sameQP_csv2 + second_largeQP2_csv2
train_csv_list3 = single_csv3 + second_largeQP1_csv3 + second_sameQP_csv3 + second_largeQP2_csv3
train_csv_list4 = single_csv4 + second_largeQP1_csv4 + second_sameQP_csv4 + second_largeQP2_csv4
train_csv_list5 = single_csv5 + second_largeQP1_csv5 + second_sameQP_csv5 + second_largeQP2_csv5
train_csv_list6 = single_csv6 + second_largeQP1_csv6 + second_sameQP_csv6 + second_largeQP2_csv6
train_csv_list7 = single_csv7 + second_largeQP1_csv7 + second_sameQP_csv7 + second_largeQP2_csv7
train_csv_list8 = single_csv8 + second_largeQP1_csv8 + second_sameQP_csv8 + second_largeQP2_csv8
train_csv_list9 = single_csv9 + second_largeQP1_csv9 + second_sameQP_csv9 + second_largeQP2_csv9
train_csv_list10 = single_csv10 + second_largeQP1_csv10 + second_sameQP_csv9 + second_largeQP2_csv10
print("train_csv_list: ", len(train_csv_list10))

train_csv_list:  600


In [10]:
def laplace_smoothing(probabilities, alpha=1):
    """
    ラプラス平滑化を行う関数
    
    Args:
    probabilities (list): 平滑化する確率分布のリスト
    alpha (float): 平滑化パラメータ
    
    Returns:
    smoothed_probabilities (list): 平滑化された確率分布のリスト
    """
    total_count = sum(probabilities)
    num_elements = len(probabilities)
    
    smoothed_probabilities = [(count + alpha) / (total_count + alpha * num_elements) for count in probabilities]
    
    return smoothed_probabilities




def process_train_csv_lists(train_csv_list):
    pu_columns = ["PU1_64", "PU1_32", "PU1_16", "PU1_8", "PU1_4",  
                  "PU2_64","PU2_32", "PU2_16", "PU2_8", "PU2_4"]

#     luminance_columns = ["LU1_0","LU1_1","LU1_2","LU1_3",
#                          "LU1_4","LU1_5","LU1_6","LU1_7",
#                          "LU1_8","LU1_9","LU1_10","LU1_11",
#                          "LU1_12","LU1_13","LU1_14","LU1_15",
#                          "LU1_16","LU1_17","LU1_18","LU1_19",
#                          "LU1_20","LU1_21","LU1_22","LU1_23",
#                          "LU1_24","LU1_25","LU1_26","LU1_27",
#                          "LU1_28","LU1_29","LU1_30","LU1_31",
#                          "LU1_32","LU1_33","LU1_34",
                         
#                          "LU2_0","LU2_1","LU2_2","LU2_3",
#                          "LU2_4","LU2_5","LU2_6","LU2_7",
#                          "LU2_8","LU2_9","LU2_10","LU2_11",
#                          "LU2_12","LU2_13","LU2_14","LU2_15",
#                          "LU2_16","LU2_17","LU2_18","LU2_19",
#                          "LU2_20","LU2_21","LU2_22","LU2_23",
#                          "LU2_24","LU2_25","LU2_26","LU2_27",
#                          "LU2_28","LU2_29","LU2_30","LU2_31",
#                          "LU2_32","LU2_33","LU2_34"]
    
    luminance_columns = ["LU1_0","LU1_1","LU1_9","LU1_10","LU1_11","LU1_25","LU1_26","LU1_27",
                         "LU2_0","LU2_1","LU2_9","LU2_10","LU2_11", "LU2_25","LU2_26","LU2_27"]

    chrominance_columns = ["CH1_0", "CH1_1", "CH1_10", "CH1_26", "CH1_34", "CH1_36", 
                           "CH2_0", "CH2_1", "CH2_10", "CH2_26", "CH2_34", "CH2_36"]
    
    
    
    label_columns = ["LABEL"]
    mae1_columns = [f"MAE1_{i}" for i in range(52)]
    mae2_columns = [f"MAE2_{i}" for i in range(52)]
    mae_columns = ["MAE"]
    final_qp_columns = ["FINAL_QP"]
    kl_divergence1 = ["KLD_PU"]
    kl_divergence2 = ["KLD_LUMA"]
    kl_divergence3 = ["KLD_CHROMA"]
    ratio_columns1 = ["RATIO1"]
    ratio_columns2 = ["RATIO2"]
    
    train_df1_1 = pd.DataFrame(columns=pu_columns)
    train_df1_2 = pd.DataFrame(columns=luminance_columns)
    train_df1_3 = pd.DataFrame(columns=chrominance_columns)
    LABEL = pd.DataFrame(columns=label_columns)
    RATIO1 = pd.DataFrame(columns=ratio_columns1)
    RATIO2 = pd.DataFrame(columns=ratio_columns2)
    train_df3 = pd.DataFrame(columns=mae1_columns)
    train_df4 = pd.DataFrame(columns=mae2_columns)
    MAE = pd.DataFrame(columns=mae_columns)
    FINAL_QP = pd.DataFrame(columns=final_qp_columns)
    kl_divergence_df1 = pd.DataFrame(columns=kl_divergence1)
    kl_divergence_df2 = pd.DataFrame(columns=kl_divergence2)
    kl_divergence_df3 = pd.DataFrame(columns=kl_divergence3)

    for path1, path2, path3, path4 in train_csv_list:
        label = 1 if ("2ndQP" in path1) and ("3rdQP" in path3) else 0
        train_pkl_list = [path2, path4]
        df1 = pd.read_csv(path1)
        df2 = pd.read_csv(path3)
        
        # 平滑化を行う
        probabilities_df1 = laplace_smoothing([df1.loc[i, "pu_counts"] for i in [0,1,2,3,4]])
        probabilities_df2 = laplace_smoothing([df2.loc[i, "pu_counts"] for i in [0,1,2,3,4]])
        kl_divergence1 = entropy(probabilities_df1, probabilities_df2)
        
        probabilities_df3 = laplace_smoothing([df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]])
        probabilities_df4 = laplace_smoothing([df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]])
        kl_divergence2 = entropy(probabilities_df3, probabilities_df4)
        
        probabilities_df5 = laplace_smoothing([df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]])
        probabilities_df6 = laplace_smoothing([df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]])
        kl_divergence3 = entropy(probabilities_df5, probabilities_df6)
        
        
        pu_values = [df1.loc[i, "pu_counts"] for i in range(5)] + [df2.loc[i, "pu_counts"] for i in range(5)]
        # lu_values = [df1.loc[i, "luminance_counts"] for i in range(35)] + [df2.loc[i, "luminance_counts"] for i in range(35)]
        lu_values = [df1.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]] + [df2.loc[i, "luminance_counts"] for i in [0,1,9,10,11,25,26,27]]
        ch_values = [df1.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]] + [df2.loc[i, "chroma_counts"] for i in [0,1,10,26,34,36]]
        
        train_df1_1 = pd.concat([train_df1_1, pd.DataFrame([pu_values], columns=pu_columns)], ignore_index=True)
        train_df1_2= pd.concat([train_df1_2, pd.DataFrame([lu_values], columns=luminance_columns)], ignore_index=True)
        train_df1_3 = pd.concat([train_df1_3, pd.DataFrame([ch_values], columns=chrominance_columns)], ignore_index=True)
        
        kl_divergence_df1 = pd.concat([kl_divergence_df1, pd.DataFrame({"KLD_PU": [kl_divergence1]})], ignore_index=True)
        kl_divergence_df2 = pd.concat([kl_divergence_df2, pd.DataFrame({"KLD_LUMA": [kl_divergence2]})], ignore_index=True)
        kl_divergence_df3 = pd.concat([kl_divergence_df3, pd.DataFrame({"KLD_CHROMA": [kl_divergence3]})], ignore_index=True)


        LABEL = pd.concat([LABEL, pd.DataFrame({"LABEL": [label]})], ignore_index=True)

        final_QP = extract_finalQP(train_pkl_list[0])

        mae_d1 = calculate_mae(train_pkl_list[0])
        mae_d2 = calculate_mae(train_pkl_list[1])
        ratio1 = ratio_double_compressed(mae_d1, final_QP)
        ratio2 = ratio_double_compressed(mae_d2, final_QP)

        RATIO1 = pd.concat([RATIO1, pd.DataFrame({"RATIO1": [ratio1]})], ignore_index=True)
        RATIO2 = pd.concat([RATIO2, pd.DataFrame({"RATIO2": [ratio2]})], ignore_index=True)

        train_df3 = pd.concat([train_df3, pd.DataFrame({f"MAE1_{i}": [mae_d1[i]] for i in range(52)})], ignore_index=True)
        train_df4 = pd.concat([train_df4, pd.DataFrame({f"MAE2_{i}": [mae_d2[i]] for i in range(52)})], ignore_index=True)
        MAE = pd.concat([MAE, pd.DataFrame({"MAE": [mae_d1]})], ignore_index=True)
        FINAL_QP = pd.concat([FINAL_QP, pd.DataFrame({"FINAL_QP": [final_QP]})], ignore_index=True)

    train_df1_1.reset_index(drop=True, inplace=True)
    train_df1_2.reset_index(drop=True, inplace=True)
    train_df1_3.reset_index(drop=True, inplace=True)
    LABEL.reset_index(drop=True, inplace=True)
    RATIO1.reset_index(drop=True, inplace=True)
    RATIO2.reset_index(drop=True, inplace=True)
    kl_divergence_df1.reset_index(drop=True, inplace=True)
    kl_divergence_df2.reset_index(drop=True, inplace=True)
    kl_divergence_df3.reset_index(drop=True, inplace=True)

    # train_df = pd.concat([train_df1_1, train_df1_2, train_df1_3, train_df3, train_df4], axis=1)
    train_df = pd.concat([FINAL_QP, train_df1_1, train_df1_2, train_df1_3, kl_divergence_df1, kl_divergence_df2, kl_divergence_df3, RATIO1, RATIO2], axis=1)
    train_df_onlyGhost = pd.concat([FINAL_QP, train_df1_1, train_df1_2, train_df1_3, RATIO1, RATIO2], axis=1)

    return train_df, train_df_onlyGhost, LABEL, MAE, FINAL_QP


In [11]:
train_df1, train_df_onlyGhost1, LABEL1, MAE1, FINAL_QP1 = process_train_csv_lists(train_csv_list1)
train_df2, train_df_onlyGhost2, LABEL2, MAE2, FINAL_QP2 = process_train_csv_lists(train_csv_list2)
train_df3, train_df_onlyGhost3, LABEL3, MAE3, FINAL_QP3 = process_train_csv_lists(train_csv_list3)
train_df4, train_df_onlyGhost4, LABEL4, MAE4, FINAL_QP4 = process_train_csv_lists(train_csv_list4)
train_df5, train_df_onlyGhost5, LABEL5, MAE5, FINAL_QP5 = process_train_csv_lists(train_csv_list5)
train_df6, train_df_onlyGhost6, LABEL6, MAE6, FINAL_QP6 = process_train_csv_lists(train_csv_list6)
train_df7, train_df_onlyGhost7, LABEL7, MAE7, FINAL_QP7 = process_train_csv_lists(train_csv_list7)
train_df8, train_df_onlyGhost8, LABEL8, MAE8, FINAL_QP8 = process_train_csv_lists(train_csv_list8)
train_df9, train_df_onlyGhost9, LABEL9, MAE9, FINAL_QP9 = process_train_csv_lists(train_csv_list9)
train_df10, train_df_onlyGhost10, LABEL10, MAE10, FINAL_QP10 = process_train_csv_lists(train_csv_list10)

In [12]:
print(train_df1)

    FINAL_QP PU1_64 PU1_32 PU1_16  PU1_8  PU1_4 PU2_64 PU2_32 PU2_16  PU2_8  PU2_4  LU1_0  LU1_1 LU1_9 LU1_10 LU1_11 LU1_25 LU1_26 LU1_27  LU2_0  LU2_1 LU2_9 LU2_10 LU2_11 LU2_25 LU2_26 LU2_27  CH1_0 CH1_1 CH1_10 CH1_26 CH1_34 CH1_36  CH2_0  CH2_1 CH2_10 CH2_26 CH2_34 CH2_36    KLD_PU  KLD_LUMA KLD_CHROMA    RATIO1    RATIO2
0         10      0   1664   7296  21524  29516      0   1664   7216  20500  30620   8494   7166  1892   4487   1803   3068   4953   2801   8470   7484  1890   4457   1778   3001   5026   2802  13784  9972   7284  10300   1968  16692  13408  10440   7620  11000   1848  15684  0.000762  0.000219   0.001361  0.048656  0.032496
1         16      0  11520  12608  17932  17940      0  11520  12544  17508  18428  10314  11188  1921   4524   1718   3104   4792   2616  10270  11704  1948   4478   1762   3060   4788   2743  13368  7640   4768   5896   1788  26540  12480   7928   5024   6188   1892  26488  0.000196  0.000294   0.000879  0.030828  0.016599
2         20      0

In [13]:
def process_results_to_lists(train_df, train_df_onlyGhost, LABEL, MAE, FINAL_QP):
    scaler = MinMaxScaler()

    # スケーラーを使って結合したデータをスケーリング
    X_train = scaler.fit_transform(train_df)
    X_train_onlyGhost = scaler.fit_transform(train_df_onlyGhost)

    # pandasをndarrayに変換
    MAE_array = MAE.values
    FINAL_QP_array = FINAL_QP.values

    # ラベルの準備
    Y_train = LABEL['LABEL'].astype(int)

    return X_train, X_train_onlyGhost, MAE_array, FINAL_QP_array, Y_train

def append_results_to_lists(train_df, train_df_onlyGhost, LABEL, MAE, FINAL_QP,
                            X_train_list, X_train_onlyGhost_list, MAE_list, FINAL_QP_list, Y_train_list):
    X_train, X_train_onlyGhost, MAE_array, FINAL_QP_array, Y_train = process_results_to_lists(train_df, train_df_onlyGhost, LABEL, MAE, FINAL_QP)
    X_train_list.append(X_train)
    X_train_onlyGhost_list.append(X_train_onlyGhost)
    MAE_list.append(MAE_array)
    FINAL_QP_list.append(FINAL_QP_array)
    Y_train_list.append(Y_train)

# リストを初期化
X_train_list = []
X_train_onlyGhost_list = []
MAE_list = []
FINAL_QP_list = []
Y_train_list = []

# データを処理してリストに追加
append_results_to_lists(train_df1, train_df_onlyGhost1, LABEL1, MAE1, FINAL_QP1, X_train_list, X_train_onlyGhost_list, MAE_list, FINAL_QP_list, Y_train_list)
append_results_to_lists(train_df2, train_df_onlyGhost2, LABEL2, MAE2, FINAL_QP2, X_train_list, X_train_onlyGhost_list, MAE_list, FINAL_QP_list, Y_train_list)
append_results_to_lists(train_df3, train_df_onlyGhost3, LABEL3, MAE3, FINAL_QP3, X_train_list, X_train_onlyGhost_list, MAE_list, FINAL_QP_list, Y_train_list)
append_results_to_lists(train_df4, train_df_onlyGhost4, LABEL4, MAE4, FINAL_QP4, X_train_list, X_train_onlyGhost_list, MAE_list, FINAL_QP_list, Y_train_list)
append_results_to_lists(train_df5, train_df_onlyGhost5, LABEL5, MAE5, FINAL_QP5, X_train_list, X_train_onlyGhost_list, MAE_list, FINAL_QP_list, Y_train_list)
append_results_to_lists(train_df6, train_df_onlyGhost6, LABEL6, MAE6, FINAL_QP6, X_train_list, X_train_onlyGhost_list, MAE_list, FINAL_QP_list, Y_train_list)
append_results_to_lists(train_df7, train_df_onlyGhost7, LABEL7, MAE7, FINAL_QP7, X_train_list, X_train_onlyGhost_list, MAE_list, FINAL_QP_list, Y_train_list)
append_results_to_lists(train_df8, train_df_onlyGhost8, LABEL8, MAE8, FINAL_QP8, X_train_list, X_train_onlyGhost_list, MAE_list, FINAL_QP_list, Y_train_list)
append_results_to_lists(train_df9, train_df_onlyGhost9, LABEL9, MAE9, FINAL_QP9, X_train_list, X_train_onlyGhost_list, MAE_list, FINAL_QP_list, Y_train_list)
append_results_to_lists(train_df10, train_df_onlyGhost10, LABEL10, MAE10, FINAL_QP10, X_train_list, X_train_onlyGhost_list, MAE_list, FINAL_QP_list, Y_train_list)

In [14]:
# Cの範囲を指定
C_values = {'C': [0.01, 0.1, 1, 10, 100, 1000, 2000, 3000, 4000, 5000]}
# C_values = {'C': [0.01, 0.1, 1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500]}
# kfold = StratifiedKFold(n_splits=10, shuffle=True)
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# 結果のデータフレームを初期化
results = pd.DataFrame(columns=['C_RBF', 'Score_RBF', 'tnr_rbf', 'tpr_rbf',
                                'C_LINEAR', 'Score_LINEAR', 'tnr_linear', 'tpr_linear',
                                'C_OG_RBF', 'Score_OG_RBF', 'tnr_og_rbf', 'tpr_og_rbf',
                                'C_OG_LINEAR', 'Score_OG_LINEAR', 'tnr_og_linear', 'tpr_og_linear',
                                'Threshold', 'Score_old', 'tnr_old', 'tpr_old'])
    
X_index = np.arange(10)  # インデックスとして0から9までの数字を用意

# ループで各分割のtrain_idsとtest_idsを取得
for fold, (train_ids, test_ids) in enumerate(kfold.split(X_index)):
    print(f"<Fold-{fold+1}>")
    print("Train indices:", train_ids)
    print("Test indices:", test_ids)
        
    test_data = [X_train_list[i] for i in test_ids]
    test_data_OG = [X_train_onlyGhost_list[i] for i in test_ids]
    test_label = [Y_train_list[i] for i in test_ids]
    
    train_data = [X_train_list[i] for i in train_ids]
    train_data_OG = [X_train_onlyGhost_list[i] for i in train_ids]
    train_label = [Y_train_list[i] for i in train_ids]
    
    
    train_data = [item for data in train_data for item in data]
    train_data_OG = [item for data in train_data_OG for item in data]
    train_label = [item for data in train_label for item in data]
    
    X_train, X_val, Y_train, Y_val = train_test_split(train_data, train_label, test_size=600, random_state=42)
    X_train_OG, X_val_OG, _, _ = train_test_split(train_data_OG, train_label, test_size=600, random_state=42)
    
    test_data = [item for data in test_data for item in data]
    test_data_OG = [item for data in test_data_OG for item in data]
    test_label = [item for data in test_label for item in data]

    MAE_data = [MAE_list[i] for i in test_ids]
    MAE_data = [item for data in MAE_data for item in data]
    
    FINAL_QP_data = [FINAL_QP_list[i] for i in test_ids]
    FINAL_QP_data = [item for data in FINAL_QP_data for item in data]
                                                                                                 
    best_threshold = 0
    best_accuracy = 0
    best_predicted_labels = []
    best_ground_truth_labels = []
                
    for threshold in np.arange(0.01, 1.00, 0.01):
        results_old = np.array([is_double_compressed(MAE_data[i], FINAL_QP_data[i], threshold) for i in range(600)])
        predicted_labels = results_old.astype(int)
        ground_truth_labels = np.array(test_label)
        accuracy = np.sum(ground_truth_labels == predicted_labels) / len(ground_truth_labels)
    
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_threshold = threshold
            best_predicted_labels = predicted_labels
            best_ground_truth_labels = ground_truth_labels
    
    best_val_score_RBF, best_svm_model_RBF, best_c_value_RBF = 0, None, None
    best_val_score_onlyGhost_RBF, best_svm_model_onlyGhost_RBF, best_c_value_onlyGhost_RBF = 0, None, None
    
    best_val_score_LINEAR, best_svm_model_LINEAR, best_c_value_LINEAR = 0, None, None
    best_val_score_onlyGhost_LINEAR, best_svm_model_onlyGhost_LINEAR, best_c_value_onlyGhost_LINEAR = 0, None, None
        
    for C_value in C_values['C']:    
        # SVMモデルのインスタンスを作成
        svm_model_RBF = SVC(kernel='rbf', C=C_value)
        svm_model_onlyGhost_RBF = SVC(kernel='rbf', C=C_value)
        
        svm_model_LINEAR = SVC(kernel='linear', C=C_value)
        svm_model_onlyGhost_LINEAR = SVC(kernel='linear', C=C_value)

        # 訓練データで訓練
        svm_model_RBF.fit(X_train, Y_train)
        svm_model_onlyGhost_RBF.fit(X_train_OG, Y_train)
        
        svm_model_LINEAR.fit(X_train, Y_train)
        svm_model_onlyGhost_LINEAR.fit(X_train_OG, Y_train)

        
        val_accuracy_RBF = accuracy_score(Y_val, svm_model_RBF.predict(X_val))
        val_accuracy_onlyGhost_RBF = accuracy_score(Y_val, svm_model_onlyGhost_RBF.predict(X_val_OG))
        
        val_accuracy_LINEAR = accuracy_score(Y_val, svm_model_LINEAR.predict(X_val))
        val_accuracy_onlyGhost_LINEAR = accuracy_score(Y_val, svm_model_onlyGhost_LINEAR.predict(X_val_OG))
        

        # 検証データでの精度が最も高かった場合、そのモデルを保存
        if val_accuracy_RBF > best_val_score_RBF:
            best_val_score_RBF, best_svm_model_RBF, best_c_value_RBF = val_accuracy_RBF, svm_model_RBF, C_value

        if val_accuracy_onlyGhost_RBF > best_val_score_onlyGhost_RBF:
            best_val_score_onlyGhost_RBF, best_svm_model_onlyGhost_RBF, best_c_value_onlyGhost_RBF = val_accuracy_onlyGhost_RBF, svm_model_onlyGhost_RBF, C_value
            
        if val_accuracy_LINEAR > best_val_score_LINEAR:
            best_val_score_LINEAR, best_svm_model_LINEAR, best_c_value_LINEAR = val_accuracy_LINEAR, svm_model_LINEAR, C_value

        if val_accuracy_onlyGhost_LINEAR > best_val_score_onlyGhost_LINEAR:
            best_val_score_onlyGhost_LINEAR, best_svm_model_onlyGhost_LINEAR, best_c_value_onlyGhost_LINEAR = val_accuracy_onlyGhost_LINEAR, svm_model_onlyGhost_LINEAR, C_value

            
    # テストデータで評価
    test_predictions_RBF = best_svm_model_RBF.predict(test_data)
    # test_predictions_prob_RBF = best_svm_model_RBF.decision_function(X_test)
    test_accuracy_RBF = accuracy_score(test_label, test_predictions_RBF)
    report_RBF = classification_report(test_label, test_predictions_RBF, digits=4)
    conf_matrix = confusion_matrix(test_label, test_predictions_RBF)
    tnr_rbf = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_rbf = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'Summary_RBF:\n{report_RBF}')
    
    test_predictions_LINEAR = best_svm_model_LINEAR.predict(test_data)
    # test_predictions_prob_LINEAR = best_svm_model_LINEAR.decision_function(X_test)
    test_accuracy_LINEAR = accuracy_score(test_label, test_predictions_LINEAR)
    report_LINEAR = classification_report(test_label, test_predictions_LINEAR, digits=4)
    conf_matrix = confusion_matrix(test_label, test_predictions_LINEAR)
    tnr_linear = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_linear = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'Summary_LINEAR:\n{report_LINEAR}')
        
    
    # テストデータで評価
    test_predictions_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.predict(test_data_OG)
    # test_predictions_prob_onlyGhost_RBF = best_svm_model_onlyGhost_RBF.decision_function(X_test_onlyGhost)
    test_accuracy_onlyGhost_RBF = accuracy_score(test_label, test_predictions_onlyGhost_RBF)
    report_onlyGhost_RBF = classification_report(test_label, test_predictions_onlyGhost_RBF, digits=4)
    conf_matrix = confusion_matrix(test_label, test_predictions_onlyGhost_RBF)
    tnr_og_rbf = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_og_rbf = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'Summary_onlyGhost_RBF:\n{report_onlyGhost_RBF}')
    
    test_predictions_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.predict(test_data_OG)
    # test_predictions_prob_onlyGhost_LINEAR = best_svm_model_onlyGhost_LINEAR.decision_function(X_test_onlyGhost)
    test_accuracy_onlyGhost_LINEAR = accuracy_score(test_label, test_predictions_onlyGhost_LINEAR)
    report_onlyGhost_LINEAR = classification_report(test_label, test_predictions_onlyGhost_LINEAR, digits=4)
    conf_matrix = confusion_matrix(test_label, test_predictions_onlyGhost_LINEAR)
    tnr_og_linear = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_og_linear = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'Summary_onlyGhost_LINEAR:\n{report_onlyGhost_LINEAR}')
    

    report_old = classification_report(best_ground_truth_labels, best_predicted_labels, labels=[0,1], target_names=['0', '1'], zero_division=0, digits=4)
    conf_matrix = confusion_matrix(best_ground_truth_labels, best_predicted_labels)
    tnr_old = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])
    tpr_old = conf_matrix[1, 1] / (conf_matrix[1, 0] + conf_matrix[1, 1])
    print(f'Summary old_model:\n{report_old}')
        
    # Test結果を保存
    
    result_row = {'C_RBF': best_c_value_RBF, 'Score_RBF': test_accuracy_RBF, 'tnr_rbf': tnr_rbf, 'tpr_rbf': tpr_rbf,
              'C_LINEAR': best_c_value_LINEAR, 'Score_LINEAR': test_accuracy_LINEAR, 'tnr_linear': tnr_linear, 'tpr_linear': tpr_linear,
              'C_OG_RBF': best_c_value_onlyGhost_RBF, 'Score_OG_RBF': test_accuracy_onlyGhost_RBF, 'tnr_og_rbf': tnr_og_rbf, 'tpr_og_rbf': tpr_og_rbf,
              'C_OG_LINEAR': best_c_value_onlyGhost_LINEAR, 'Score_OG_LINEAR': test_accuracy_onlyGhost_LINEAR, 'tnr_og_linear': tnr_og_linear, 'tpr_og_linear': tpr_og_linear,
              'Threshold': best_threshold, 'Score_old': best_accuracy, 'tnr_old': tnr_old, 'tpr_old': tpr_old,}


    results = pd.concat([results, pd.DataFrame([result_row])], ignore_index=True)


<Fold-1>
Train indices: [0 1 2 3 4 5 6 7 9]
Test indices: [8]
Summary_RBF:
              precision    recall  f1-score   support

           0     0.7855    0.7567    0.7708       300
           1     0.7653    0.7933    0.7791       300

    accuracy                         0.7750       600
   macro avg     0.7754    0.7750    0.7749       600
weighted avg     0.7754    0.7750    0.7749       600

Summary_LINEAR:
              precision    recall  f1-score   support

           0     0.8057    0.6633    0.7276       300
           1     0.7139    0.8400    0.7718       300

    accuracy                         0.7517       600
   macro avg     0.7598    0.7517    0.7497       600
weighted avg     0.7598    0.7517    0.7497       600

Summary_onlyGhost_RBF:
              precision    recall  f1-score   support

           0     0.7444    0.6600    0.6996       300
           1     0.6946    0.7733    0.7319       300

    accuracy                         0.7167       600
   macro avg  

In [15]:
# 各統計情報を100倍して小数点第4位までの表記に変更
statistics_data = {
    'Model': ['RBF', 'LINEAR', 'Only Ghost and RBF', 'Only Ghost and LINEAR', 'OLD'],
    'Average TNR': [round(results['tnr_rbf'].mean() * 100, 2), round(results['tnr_linear'].mean() * 100, 2), round(results['tnr_og_rbf'].mean() * 100, 2), round(results['tnr_og_linear'].mean() * 100, 2), round(results['tnr_old'].mean() * 100, 2)],
    'Average TPR': [round(results['tpr_rbf'].mean() * 100, 2), round(results['tpr_linear'].mean() * 100, 2), round(results['tpr_og_rbf'].mean() * 100, 2), round(results['tpr_og_linear'].mean() * 100, 2), round(results['tpr_old'].mean() * 100, 2)],
    'Average Test Score': [round(results['Score_RBF'].mean() * 100, 2), round(results['Score_LINEAR'].mean() * 100, 2), round(results['Score_OG_RBF'].mean() * 100, 2), round(results['Score_OG_LINEAR'].mean() * 100, 2), round(results['Score_old'].mean() * 100, 2)],
    'Standard Deviation': [round(results['Score_RBF'].std() * 100, 2), round(results['Score_LINEAR'].std() * 100, 2), round(results['Score_OG_RBF'].std() * 100, 2), round(results['Score_OG_LINEAR'].std() * 100, 2), round(results['Score_old'].std() * 100, 2)],
    'Max Test Score': [round(results['Score_RBF'].max() * 100, 2), round(results['Score_LINEAR'].max() * 100, 2), round(results['Score_OG_RBF'].max() * 100, 2), round(results['Score_OG_LINEAR'].max() * 100, 2), round(results['Score_old'].max() * 100, 2)],
    'Min Test Score': [round(results['Score_RBF'].min() * 100, 2), round(results['Score_LINEAR'].min() * 100, 2), round(results['Score_OG_RBF'].min() * 100, 2), round(results['Score_OG_LINEAR'].min() * 100, 2), round(results['Score_old'].min() * 100, 2)],
}

# DataFrameを作成
statistics_df = pd.DataFrame(statistics_data)

# 表示
print(statistics_df)

                   Model  Average TNR  Average TPR  Average Test Score  Standard Deviation  Max Test Score  Min Test Score
0                    RBF        75.63        72.93               74.28                5.88           80.33           61.83
1                 LINEAR        73.30        71.83               72.57                5.51           79.17           62.17
2     Only Ghost and RBF        72.10        71.03               71.57                6.55           78.17           59.50
3  Only Ghost and LINEAR        71.73        71.13               71.43                6.02           78.83           60.17
4                    OLD        92.70        25.80               59.25                0.82           60.50           58.00


In [16]:
print(results['C_RBF'])
print(results['C_LINEAR'])

0    100
1    100
2    100
3    100
4    100
5    100
6     10
7    100
8    100
9     10
Name: C_RBF, dtype: object
0     100
1      10
2    1000
3      10
4     100
5    1000
6      10
7     100
8    4000
9      10
Name: C_LINEAR, dtype: object


In [17]:
statistics_df.to_csv('statistics_data10.csv', index=False)